# Layers and Modules

A **module is a set of layers**. As such, it could be a single layer, a generic subset of the model or the entire model.\
Each module must have a forward and a backpropagation method. It may, or may not, have parameters inside itself.\
Since auto-differentiation provides us with backpropagation, we are only intersted about parameters and forward propagation.

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

Let's revisit the code for implementing MLPs.

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

# A custom module

Let's create a module with a hidden layer of 256 hidden units, and a 10-dimensional output layer. We modify the parent's class only in order to provide the essential features that a layer must give.

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

torch.Size([2, 10])

## Sequential Class

It essentially contains a method for appending modules to the list, and a method to do the forward pass.

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)

        return X

In [7]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

## Customizing the forward propagation method

Let's implement a function that calculate a function of the type

$$f(x, w) = c \cdot \mathbf{w}^T\mathbf{x}$$

In [8]:
class FixedHiddenMlp(nn.Module):
    def __init__(self):
        super().__init__()
        #Random weight parameters that will not compute gradients and 
        #therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        #Reuse the fully connected layer. This is equivalent to sharing
        #parameters with two fully connected layers
        X = self.linear(X)
        #Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

Note how the layer weights are (randomly) intialized as constants, thus they are not updated with backpropagation.

In [9]:
net = FixedHiddenMlp()
net(X)

tensor(0.1654, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMlp())
chimera(X)

tensor(0.0587, grad_fn=<SumBackward0>)

# Parameter Management


## Accessing parameters

In [11]:
import torch
from torch import nn

In [12]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In `Sequential`, we can access parameters as if they were elements of a list.

In [13]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0067,  0.1554, -0.2539, -0.2265, -0.0003,  0.1152, -0.0913, -0.0853]])),
             ('bias', tensor([-0.2261]))])

In [14]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.2261]))

Gradients are not initialized until their backpropagation method isn't called.

In [15]:
net[2].weight.grad == None

True

An example of an operation on a single parameter.

In [16]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## Sharing parameters

In [17]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
#Let's check that the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
#Make sure they are actually the same object rather than just having the same value
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


The gradients of equal subsequent layers are added together during backpropagation.

# Parameter initialization

Usually initialization is peformed at random. We could want to change this according to our needs.

In [18]:
import torch
from torch import nn

However, there is a variety of available present initialization methods.

In [19]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size = (2, 4))
net(X).shape

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

# Built-in initialization

In [20]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean = 0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0048,  0.0212, -0.0047,  0.0053]), tensor(0.))

In [21]:
# Initializing all the parameters to a given costant value

def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [23]:
# Different types of initializations per block

def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([-0.4229,  0.2163,  0.0484, -0.5328])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


## Initialization with a certain distribution

In [34]:
def my_init(module):
    if type(module) == nn.Linear:
        print('Init', *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  5.3219, -0.0000,  5.2742],
        [-8.0620,  6.5694,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

## Initializing parameters directly

In [37]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([ 42.0000, 107.3219, 102.0000, 107.2742])

# Lazy initialization

When lazy initialization is employed, the framework defers initializaition until the very first moment the model is run.

In [38]:
import torch
from torch import nn
from d2l import torch as d2l

In [39]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Note how parameters are not yet initialized.

In [40]:
net[0].weight

<UninitializedParameter>

In [41]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

Let's build a method to initialize parameters with a dry run of the model.

In [42]:
@d2l.add_to_class(d2l.Module)   #@save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

# Custom Layers

In [43]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## Layers without parameters

In [44]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [46]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [47]:
# Incorporating the centered layer

net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [48]:
Y = net(torch.rand((4, 8)))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

## Layers with parameters

There are built-in functions to create parameters, which are useful to housekeep them during training, initialization, saving, etc.

In [49]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [50]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.1232, -0.1936,  1.2670],
        [-1.5120,  0.7545, -0.5281],
        [-0.4066, -1.3268,  0.7022],
        [-0.6382,  0.7582,  0.4773],
        [-1.3652, -0.2629,  1.0977]], requires_grad=True)

In [51]:
linear(torch.rand((2, 5)))

tensor([[0.0000, 0.0888, 2.3767],
        [0.0000, 0.0000, 2.4787]])

In [52]:
# An example of network with custom layers

net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[6.7267],
        [4.0196]])

# File I/O

It is useful, as a backup and a metric, to know how to load and store individual weight vectors and entire models.

In [53]:
import torch
from torch import nn
from torch.nn import functional as F

## Loading and saving tensors

In [54]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [55]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [56]:
# Storing a list of tensors
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [57]:
# Dictionary for mapping from string to tensor

mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## Loading and saving model parameters

Note that this saves exactly and only the parameters, not the architecture as well.

In [58]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
    
net = MLP()
X = torch.randn((2, 20))
Y = net(X)

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [59]:
# The method

torch.save(net.state_dict(), 'mlp.params')

In [60]:
# Recovering the MLP model

clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

Since they are basically the same model, let's verify that they bear the same outputs.

In [61]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# GPUs

Normally, in PyTorch each array has a device: it might be the same CPU or multiple GPUs across different servers.